In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import math

# Load the MoveNet model from TF Hub
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# Define the EDGES dictionary
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def getAngle(pt1, pt2, pt3):
    dx1 = pt2[0] - pt1[0]
    dy1 = pt2[1] - pt1[1]
    dx2 = pt3[0] - pt1[0]
    dy2 = pt3[1] - pt1[1]
    
    angle = math.atan2(dy2, dx2) - math.atan2(dy1, dx1)
    angle = math.degrees(angle)
    
    if angle < 0:
        angle += 360
    
    if angle > 180:
        angle = 360 - angle
    
    return angle

keypoint_radius = 2
line_width = 1

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), keypoint_radius, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            angle = getAngle((x1, y1), (x2, y2), shaped[7])
            if angle > 45:
                cv2.line_color = (0, 255, 0)  # Change line color to green for angles greater than 45
            else:
                cv2.line_color = (0, 0, 255)  # Default to red color
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), cv2.line_color, line_width)

# Open the video file
cap = cv2.VideoCapture('footballers.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384, 640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    for person in keypoints_with_scores:
        draw_connections(frame, person, EDGES, 0.1)
        draw_keypoints(frame, person, 0.1)
    
    cv2.imshow('Multi-Person Pose Estimation', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

